In [202]:
import pandas as pd
import numpy as np
import math

In [203]:
# Load CSV
file_path = "output/step-2-weekly-highs-and-lows.csv"
df = pd.read_csv(file_path)
df.columns = df.columns.str.strip()
df.tail()

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close,Day,Week,Year,Order Type,Buy Quantity,Investment,Total Share,Total Investment,Average Price,Target,IRLower,IRUpper,Description
294,2023-09-26,"2,491.00","2,499.00","2,452.60","2,482.10","2,457.95",Tuesday,39,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,2023-09-27,"2,460.00","2,498.00","2,445.10","2,457.95","2,483.30",Wednesday,39,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,2023-09-28,"2,489.95","2,511.00","2,446.70","2,483.30","2,475.25",Thursday,39,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,2023-09-29,"2,462.05","2,479.55","2,405.05","2,475.25","2,413.90",Friday,39,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,NaN,NaN,"2,511.00","2,405.05",NaN,NaN,WEEKLY SUMMARY,39,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
# Columns to clean
cols_to_convert = ["OPEN", "HIGH", "LOW", "PREV. CLOSE", "close"]

# Convert columns: Remove commas and change to float
for col in cols_to_convert:
    df[col] = df[col].astype(str).str.replace(',', '', regex=True).astype(float)

df.head()

,Date,OPEN,HIGH,LOW,PREV. CLOSE,close,Day,Week,Year,Order Type,Buy Quantity,Investment,Total Share,Total Investment,Average Price,Target,IRLower,IRUpper,Description
0,2022-10-03,3440.0,3460.05,3110.20,3455.75,3157.3,Monday,40,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-10-04,3230.0,3291.20,3125.00,3157.30,3237.7,Tuesday,40,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-10-06,3270.0,3318.80,3208.05,3237.70,3285.0,Thursday,40,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-10-07,3280.0,3310.95,3228.15,3285.00,3277.6,Friday,40,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,3460.05,3110.20,NaN,NaN,WEEKLY SUMMARY,40,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [205]:
INVESTMENT_PER_TRADE = 5000
TARGET_MULTIPLIER = 1.06
UPPER_MULTIPLIER = 1.0275
LOWER_MULTIPLIER = 0.9725

In [206]:
# Initialize tracking variables
total_shares = 0
total_investment = 0
avg_price = 0
prev_week_high = None
active_buys = []

updated_rows = []
weekIdx = 0
prev_year = None

# Process year-wise and then week-wise
for (year, week), group in df.groupby(['Year', 'Week']):
    
    # Reset tracking variables when the year changes
    if prev_year is None or prev_year != year:
        prev_week_high = None
        weekIdx = 0  # Reset week index for new year
    
    weekIdx += 1
    weekly_high = group['HIGH'].max()
    weekly_low = group['LOW'].min()

    can_buy_this_week = True
    
    for _, row in group.iterrows():
        if weekIdx == 1:
            prev_week_high = weekly_high
            updated_rows.append({**row})
            continue  # Skip first week of each year

        if row['Day'] == "WEEKLY SUMMARY":
            updated_rows.append({ **row })
            continue
        
        # Sell condition
        if total_shares > 0 and row['HIGH'] >= avg_price * TARGET_MULTIPLIER:
            sell_price = avg_price * TARGET_MULTIPLIER
            profit = (sell_price - avg_price) * total_shares

            updated_rows.append({
                **row,
                "Order Type": "SELL",
                "Buy Quantity": "",
                "Investment": "",
                "Total Share": 0,
                "Total Investment": 0,
                "Average Price": "",
                "Target": "",
                "IRLower": "",
                "IRUpper": "",
                "Description": f"Sold {total_shares} shares at {sell_price}, Profit: {profit}",
                "Profit": profit,
                "Capital Employed": total_shares * sell_price,
                "Profit Percentage": (profit / (total_shares * avg_price)) * 100 
            })

            total_shares = 0
            total_investment = 0
            avg_price = 0
            active_buys = []
            continue

        if can_buy_this_week:
            # Determine buy price
            if total_shares == 0:
                if row['LOW'] <= prev_week_high and prev_week_high <= row['HIGH']: 
                    buy_price = prev_week_high
                    buy_reason = "Prev week high - Buy price"
                else:
                    updated_rows.append({
                        **row,
                        "Order Type": "NOTHING",
                    })
                    continue
            else:
                if row['LOW'] <= LOWER_MULTIPLIER * avg_price:
                    buy_price = LOWER_MULTIPLIER * avg_price
                    buy_reason = "LOWER - Buy Triggered at IRLower"
                elif row['HIGH'] >= UPPER_MULTIPLIER * avg_price:
                    buy_price = UPPER_MULTIPLIER * avg_price
                    buy_reason = "UPPER - Buy Triggered at IRUpper"
                else:
                    updated_rows.append({**row, buy_reason: "OUT_OF_RANGE"})
                    continue
            # HOTFIX: Buy price is always max of last week
            buy_price = prev_week_high
            can_buy_this_week = False
            
            buy_quantity = math.floor(INVESTMENT_PER_TRADE / buy_price)
            investment = buy_quantity * buy_price
    
            total_shares += buy_quantity
            total_investment += investment
            avg_price = total_investment / total_shares
    
            target_price = avg_price * TARGET_MULTIPLIER
            upper_range = avg_price * UPPER_MULTIPLIER
            lower_range = avg_price * LOWER_MULTIPLIER
    
            
            active_buys.append({
                "Order Type": "BUY",
                "Year": year,
                "Week": week,
                "Buy Price": buy_price,
                "Buy Quantity": buy_quantity,
                "Investment": investment,
                "Total Shares": total_shares,
                "Total Investment": total_investment,
                "Avg Price": avg_price,
                "Target": target_price,
                "IRLower": lower_range,
                "IRUpper": upper_range,
                "Description": buy_reason
            })
            
            updated_rows.append({
                **row,
                "Order Type": "BUY",
                "Buy Quantity": buy_quantity,
                "Investment": investment,
                "Total Share": total_shares,
                "Total Investment": total_investment,
                "Average Price": avg_price,
                "Target": target_price,
                "IRLower": lower_range,
                "IRUpper": upper_range,
                "Description": buy_reason
            })
        else:
            updated_rows.append({**row, "Order Type": "NOTHING", "Description": "One buy per week rule"})
            
    prev_week_high = weekly_high
    prev_year = year  # Update year tracker

# Create new DataFrame with updated data
df_updated = pd.DataFrame(updated_rows)
df = df_updated

In [207]:
columns_to_round = ['Buy Quantity', 'Investment', 'Total Share', 'Total Investment',
                    'Average Price', 'Target', 'IRLower', 'IRUpper']

df[columns_to_round] = df[columns_to_round].apply(
       lambda x: pd.to_numeric(x, errors='coerce').round(3)
)

df.to_csv("output/step-3-buy-orders.csv")
print("All trade signals processing done.")

All trade signals processing done.
